In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=bce8ad9ddcdc2ab1eb1ad27fc4efe0de930e648741d92b08cc670986cdd8555f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!pip install dtreeviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.4 MB/s eta 0:00:00


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import expr

import dtreeviz

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/04 09:23:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Data:

### 1.1. Read Data:

In [4]:
# read csv:
df = spark.read.csv('/kaggle/input/big-data/PS_20174392719_1491204439457_log.csv', inferSchema=True, header=True)

In [5]:
df = df.select("type", "amount", "oldbalanceOrg", "newbalanceOrig", "isFraud")

In [6]:
df.show(2)

+-------+-------+-------------+--------------+-------+
|   type| amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+-------+-------------+--------------+-------+
|PAYMENT|9839.64|     170136.0|     160296.36|      0|
|PAYMENT|1864.28|      21249.0|      19384.72|      0|
+-------+-------+-------------+--------------+-------+
only showing top 2 rows



### 1.2. Data Preparation:

In [7]:
# train test split:
train, test = df.randomSplit([0.7, 0.3], seed = 7)

In [8]:
print(f"Train set length: {train.count()} records")
print(f"Test set length: {test.count()} records")

Train set length: 4451877 records


Test set length: 1910743 records


In [9]:
train.show(2)

+-------+------+-------------+--------------+-------+
|   type|amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+------+-------------+--------------+-------+
|CASH_IN|  1.42|   1270713.41|    1270714.83|      0|
|CASH_IN|  4.35|   4136277.22|    4136281.57|      0|
+-------+------+-------------+--------------+-------+
only showing top 2 rows



In [10]:
# dtype:
train.dtypes

[('type', 'string'),
 ('amount', 'double'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('isFraud', 'int')]

In [11]:
catCols = [x for (x, dataType) in train.dtypes if dataType == 'string']
numCols = [x for (x, dataType) in train.dtypes if ((dataType == 'double') & (x != 'isFraud'))]

In [12]:
numCols

['amount', 'oldbalanceOrg', 'newbalanceOrig']

In [13]:
catCols

['type']

In [14]:
# One hot Encoding:
train.agg(F.countDistinct('type')).show()

24/05/04 09:24:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 09:24:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 09:24:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 09:24:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------------------+
|count(DISTINCT type)|
+--------------------+
|                   5|
+--------------------+



In [15]:
train.groupBy('type').count().show()

24/05/04 09:24:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 09:24:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 09:24:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 09:24:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 372791|
| CASH_IN| 979384|
|CASH_OUT|1565928|
| PAYMENT|1504894|
|   DEBIT|  28880|
+--------+-------+



In [16]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer 

In [17]:
string_indexer = [StringIndexer(inputCol = x, outputCol = x + "_StringIndexer", handleInvalid="skip") for x in catCols]

In [18]:
one_hot_encoder = [OneHotEncoder(
                        inputCols=[f"{x}_StringIndexer" for x in catCols],
                        outputCols=[f"{x}_OneHotEncoder" for x in catCols],
    )
]

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
assemblerInput = [x for x in numCols]
assemblerInput += [f"{x}_OneHotEncoder"for x in catCols]

In [21]:
assemblerInput

['amount', 'oldbalanceOrg', 'newbalanceOrig', 'type_OneHotEncoder']

In [22]:
vector_assembler = VectorAssembler(inputCols = assemblerInput, outputCol = "VectorAssembler_features")

In [23]:
stages = []
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]

In [24]:
stages

[StringIndexer_c79e3ffcb2ee,
 OneHotEncoder_6e89b5cb5bd8,
 VectorAssembler_b0a2fe60295a]

In [25]:
%%time
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages(stages)
model = pipeline.fit(train)

CPU times: user 28.5 ms, sys: 10.7 ms, total: 39.2 ms
Wall time: 20 s


In [26]:
pp_df = model.transform(test)
pp_df.select("type", "amount", "oldbalanceOrg", "newbalanceOrig", "VectorAssembler_features").show(truncate=False)

+-------+------+-------------+--------------+---------------------------------------------------+
|type   |amount|oldbalanceOrg|newbalanceOrig|VectorAssembler_features                           |
+-------+------+-------------+--------------+---------------------------------------------------+
|CASH_IN|4.58  |94241.0      |94245.58      |[4.58,94241.0,94245.58,0.0,0.0,1.0,0.0]            |
|CASH_IN|5.44  |0.0          |5.44          |(7,[0,2,5],[5.44,5.44,1.0])                        |
|CASH_IN|6.07  |400680.0     |400686.07     |[6.07,400680.0,400686.07,0.0,0.0,1.0,0.0]          |
|CASH_IN|6.76  |11322.0      |11328.76      |[6.76,11322.0,11328.76,0.0,0.0,1.0,0.0]            |
|CASH_IN|8.27  |8428410.94   |8428419.21    |[8.27,8428410.94,8428419.21,0.0,0.0,1.0,0.0]       |
|CASH_IN|8.44  |39384.0      |39392.44      |[8.44,39384.0,39392.44,0.0,0.0,1.0,0.0]            |
|CASH_IN|11.13 |4116439.74   |4116450.87    |[11.13,4116439.74,4116450.87,0.0,0.0,1.0,0.0]      |
|CASH_IN|12.79 |6017

## 2. Model:

### 1. Logistic Regression:

In [27]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier

In [28]:
data = pp_df.select(
    F.col("VectorAssembler_features").alias("features"),
    F.col("isFraud").alias('label'))

In [29]:
data.show(5, truncate = False)

+--------------------------------------------+-----+
|features                                    |label|
+--------------------------------------------+-----+
|[4.58,94241.0,94245.58,0.0,0.0,1.0,0.0]     |0    |
|(7,[0,2,5],[5.44,5.44,1.0])                 |0    |
|[6.07,400680.0,400686.07,0.0,0.0,1.0,0.0]   |0    |
|[6.76,11322.0,11328.76,0.0,0.0,1.0,0.0]     |0    |
|[8.27,8428410.94,8428419.21,0.0,0.0,1.0,0.0]|0    |
+--------------------------------------------+-----+
only showing top 5 rows



In [30]:
%%time
model = LogisticRegression().fit(data)

24/05/04 09:26:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


CPU times: user 47.2 ms, sys: 15.1 ms, total: 62.2 ms
Wall time: 58.6 s


In [31]:
model.summary.areaUnderROC

0.992874229305281

In [32]:
model.summary.accuracy

0.9991118638142336

In [33]:
model.summary.pr.show()

+------------------+-------------------+
|            recall|          precision|
+------------------+-------------------+
|               0.0| 0.9127789046653144|
|0.3598560575769692| 0.9127789046653144|
|0.4838064774090364| 0.6813063063063063|
|0.5501799280287885| 0.5362431800467654|
| 0.594562175129948| 0.4430870083432658|
|0.6309476209516194|0.38060781476121563|
|0.6613354658136745|0.33508914100486226|
|0.6889244302279088| 0.3009081383164513|
|0.7113154738104758|0.27302025782688766|
|0.7305077968812475|0.25006843690117714|
|0.7477009196321471|0.23097826086956522|
|0.7616953218712516|0.21438217420661715|
|0.7752898840463814| 0.2003927242662257|
|0.7860855657736905|0.18784635964074145|
|0.7968812475009996|0.17706112295664533|
|0.8000799680127949|0.16611323260833472|
|0.8036785285885646|0.15659083826737302|
|0.8096761295481807|0.14861294583883752|
|0.8128748500599761|0.14102386237513873|
|0.8164734106357457| 0.1342890964093121|
+------------------+-------------------+
only showing top

In [34]:
# Save the model
model.save("Log_model")

### 2. Decision Tree:

In [35]:
%%time
model = DecisionTreeClassifier().fit(data)

CPU times: user 72.7 ms, sys: 22.7 ms, total: 95.3 ms
Wall time: 1min 33s


In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:
predictions = model.transform(data)

In [38]:
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

In [39]:
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 1.00


In [40]:
accuracy

0.9990396406005413

In [41]:
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_6024db32a8e0, depth=5, numNodes=25, numClasses=2, numFeatures=7
  If (feature 0 <= 1441000.21)
   If (feature 6 in {0.0})
    If (feature 0 <= 539091.01)
     Predict: 0.0
    Else (feature 0 > 539091.01)
     If (feature 1 <= 350647.275)
      Predict: 0.0
     Else (feature 1 > 350647.275)
      If (feature 2 <= 17.475)
       Predict: 1.0
      Else (feature 2 > 17.475)
       Predict: 0.0
   Else (feature 6 not in {0.0})
    If (feature 1 <= 708756.63)
     Predict: 0.0
    Else (feature 1 > 708756.63)
     If (feature 2 <= 17.475)
      Predict: 1.0
     Else (feature 2 > 17.475)
      Predict: 0.0
  Else (feature 0 > 1441000.21)
   If (feature 1 <= 708756.63)
    Predict: 0.0
   Else (feature 1 > 708756.63)
    If (feature 1 <= 2060406.775)
     If (feature 3 in {1.0})
      Predict: 1.0
     Else (feature 3 not in {1.0})
      Predict: 0.0
    Else (feature 1 > 2060406.775)
     If (feature 2 <= 17.475)
      Predict: 1

In [42]:
# Save the model
model.save("Dtree_model")

In [43]:
# Load the model
from pyspark.ml.classification import DecisionTreeClassificationModel
dtree_model = DecisionTreeClassificationModel.load("Dtree_model")

## HAPPY ENDING